<a href="https://colab.research.google.com/github/cyberanalyst268/Data-Science-and-Decision-Making/blob/main/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import numpy as np
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt
np.set_printoptions(precision=3)

In [18]:
#Load Jester dataset
data = pd.read_csv('https://raw.githubusercontent.com/albanda/CE888/master/lab4-recommender/jester-data-1.csv', na_values=["?"],header=None)  
data = data.replace(99, np.NaN)
data = data.drop(data.columns[0], axis=1)
data

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,...,2.82,NaN,NaN,NaN,NaN,NaN,-5.63,NaN,NaN,NaN
1,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,...,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,NaN,NaN,NaN,NaN,9.03,9.27,9.03,9.27,NaN,NaN,...,NaN,NaN,NaN,9.08,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,8.35,NaN,NaN,1.80,8.16,-2.82,6.21,NaN,1.84,...,NaN,NaN,NaN,0.53,NaN,NaN,NaN,NaN,NaN,NaN
4,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,...,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24978,0.44,7.43,9.08,2.33,3.20,6.75,-8.79,-0.53,-8.74,7.23,...,8.83,-1.21,9.22,-6.70,8.45,9.03,6.55,8.69,8.79,7.43
24979,9.13,-8.16,8.59,9.08,0.87,-8.93,-3.50,5.78,-8.11,4.90,...,-1.17,-5.73,-1.46,0.24,9.22,-8.20,-7.23,-8.59,9.13,8.45
24980,NaN,NaN,NaN,NaN,-7.77,NaN,6.70,-6.75,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24981,NaN,NaN,NaN,NaN,-9.71,NaN,4.56,-8.30,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
data.idxmax(axis="columns")

0        29
1        10
2        13
3         2
4        53
         ..
24978    64
24979    39
24980    51
24981    29
24982    14
Length: 24983, dtype: int64

In [20]:
data.idxmin(axis="columns")

0         7
1         6
2        17
3        38
4        16
         ..
24978    87
24979    28
24980    35
24981     5
24982    19
Length: 24983, dtype: int64

In [21]:
arr = data.values
print(arr.shape)

(24983, 100)


In [22]:
rated = np.where(arr!=99)
print(len(rated[0]), rated[1].shape)

2498300 (2498300,)


In [23]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [24]:
new_arr, idx = replace(arr, 0.1)

In [25]:
arr[idx[0][0], idx[1][0]]

nan

In [26]:
new_arr[idx[0][0], idx[1][0]]

99.0

In [27]:
n_latent_factors = 2

user_ratings = data.values
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [28]:
latent_item_features

array([[0.169, 0.871],
       [0.131, 0.145],
       [0.547, 0.929],
       [0.235, 0.448],
       [0.295, 0.562],
       [0.477, 0.111],
       [0.65 , 0.474],
       [0.206, 0.858],
       [0.778, 0.375],
       [0.092, 0.699],
       [0.392, 0.654],
       [0.456, 0.39 ],
       [0.527, 0.61 ],
       [0.827, 0.76 ],
       [0.929, 0.073],
       [0.676, 0.848],
       [0.667, 0.117],
       [0.192, 0.863],
       [0.031, 0.322],
       [0.788, 0.81 ],
       [0.623, 0.574],
       [0.297, 0.522],
       [0.733, 0.727],
       [0.248, 0.764],
       [0.388, 0.453],
       [0.111, 0.758],
       [0.201, 0.581],
       [0.152, 0.626],
       [0.171, 0.939],
       [0.219, 0.208],
       [0.753, 0.66 ],
       [0.785, 0.663],
       [0.799, 0.803],
       [0.693, 0.033],
       [0.105, 0.343],
       [0.471, 0.486],
       [0.212, 0.296],
       [0.582, 0.353],
       [0.424, 0.469],
       [0.906, 0.632],
       [0.005, 0.086],
       [0.655, 0.308],
       [0.24 , 0.684],
       [0.1

In [29]:
latent_user_preferences

array([[0.522, 0.529],
       [0.9  , 0.314],
       [0.517, 0.155],
       ...,
       [0.984, 0.639],
       [0.316, 0.6  ],
       [0.476, 0.447]])

In [32]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    #print(item_id)
    predicted_rating = predict_rating(user_id, item_id)
    err =  predicted_rating - rating
    #print(err)
    user_pref_values = latent_user_preferences[user_id]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    mse_history = []
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id, item_id]
                if not np.isnan(rating):
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 1) == 0:
            print('Iteration %d/%d:\tMSE=%.6f' % (iteration, iterations, mse))
            mse_history.append(mse)
    return mse_history

In [34]:
num_iter = 10
hist = sgd(num_iter)  # Note how the MSE decreases with the number of iterations
plt.figure()
plt.plot(np.arange(0, num_iter, 1), hist)
plt.xlabel("Iterations")
plt.ylabel("MSE")
plt.show()

Iteration 0/10:	MSE=25.267343
Iteration 1/10:	MSE=24.966224
Iteration 2/10:	MSE=24.728640


KeyboardInterrupt: ignored

In [ ]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

In [ ]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = data.columns

In [ ]:
comparison_data